In [ ]:
import numpy as np

# Implementación de la red neuronal con retropropagación

class NetNode(object):

  def __init__(self):
    self.inputs = []
    self.weights = []
    self.value = None

class Network(object):

  def __init__(self, layers, activation='relu', regression=False):
    self.net = [[NetNode() for _ in range(size)] for size in layers]
    sizes = len(layers)
    for layer in range(1, sizes):
        for node in self.net[layer]:
            for unit in self.net[layer - 1]:
                node.inputs.append(unit)
                node.weights.append(0)
            node.weights = [np.random.uniform() for _ in range(len(node.weights))]
    self.regression = regression
    if activation == 'relu':
        self.activation_func = self.relu
        self.activation_func_prime = self.relu_prime
    else:
        self.activation_func = self.sigmoide
        self.activation_func_prime = self.sigmoide_prime

  def accuracy(self, examples):
    correct = 0
    error = 0
    total = 0
    for x_test, y_test in examples:
      prediction = self.predict(x_test)
      if self.regression:
        index = 0
        for value in prediction:
          error += (y_test[index] - value) ** 2
          index += 1
          total += 1
      else:
        if (y_test[prediction] == 1):
          correct += 1
    if self.regression:
      return 1 - error / total
    else:
      return correct / len(examples)

  def backpropagation(self, eta, examples, epochs):
    inputs = self.net[0]
    outputs = self.net[-1]
    layer_size = len(self.net)
    for epoch in range(epochs):
        for x_train, y_train in examples:
            for value, node in zip(x_train, inputs):
                node.value = value
            for layer in self.net[1:]:
                for node in layer:
                    in_val = [n.value for n in node.inputs]
                    unit_value = np.dot(in_val, node.weights)
                    node.value = self.activation_func(unit_value)
            delta = [[] for _ in range(layer_size)]
            err = [y_train[i] - outputs[i].value for i in range(len(outputs))]
            delta[-1] = [self.activation_func_prime(outputs[i].value) * err[i] for i in range(len(outputs))]
            hidden_layers = layer_size - 2
            for i in range(hidden_layers, 0, -1):
                layer = self.net[i]
                n_layers = len(layer)
                w = [[node.weights[l] for node in self.net[i + 1]] for l in range(n_layers)]
                delta[i] = [self.activation_func_prime(layer[j].value) * np.dot(w[j], delta[i + 1]) for j in range(n_layers)]
            for i in range(1, layer_size):
                layer = self.net[i]
                in_val = [node.value for node in self.net[i - 1]]
                n_layers = len(self.net[i])
                for j in range(n_layers):
                    layer[j].weights = np.add(layer[j].weights, np.multiply(eta * delta[i][j], in_val))
        print(f"epoch {epoch}/{epochs} | total error={np.sum(err)/len(examples)}")
    nodes = []
    for layer in self.net[1:]:
      for node in layer:
        nodes.append(node.value)
    return nodes
  
  def predict(self, input_data):
    inputs = self.net[0]
    for v, n in zip(input_data, inputs):
        n.value = v
    for layer in self.net[1:]:
        for node in layer:
            in_val = [n.value for n in node.inputs]
            unit_value = np.dot(in_val, node.weights)
            node.value = self.activation_func(unit_value)
    if self.regression:
      outputs = []
      for node in self.net[-1]:
        outputs.append(node.value)
      return outputs
    else:
      outputs = self.net[-1]
      return outputs.index(max(outputs, key=lambda node: node.value))

  def relu(self, z):
    print(z)
    return max(0, z)

  def relu_prime(self, z):
    return 1 if z > 0 else 0

  def sigmoide(self, z):
    return 1 / (1 + np.exp(-z))

  def sigmoide_prime(self, z):
    s = self.sigmoide(z)
    return s * (1 - s)
  
  def set_weights(self, weights_list):
    idx = 0
    for layer in self.net[1:]:
        for node in layer:
          if len(node.weights) == len(weights_list[idx]):
            node.weights = weights_list[idx]
          idx += 1

  def weights(self):
    all_weights = []
    for layer in self.net[1:]:
        for node in layer:
            all_weights.append(node.weights)
    return all_weights

# Usando la red neuronal con un dataset

from sklearn import datasets
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

iris_X, iris_y = datasets.load_iris(return_X_y=True)

iris_x_normalized = normalize(iris_X, axis=0)

X_train, X_test, y_train, y_test = train_test_split(iris_x_normalized, iris_y, test_size=0.2, shuffle=True)

y_train = np_utils.to_categorical(y_train, num_classes=3)
y_test = np_utils.to_categorical(y_test, num_classes=3)

examples = []
for i in range(len(X_train)):
    examples.append([X_train[i], y_train[i]])

net = Network([4, 7, 3], regression=True)
net.backpropagation(0.1, examples, 500)

#precisión alcanzada con los datos de entrenamiento
accuracy = net.accuracy(examples)
print(f"Accuracy: {accuracy}")

#precisión alcanzada con los datos de prueba
examples = []
for i in range(len(X_test)):
    examples.append([X_test[i], y_test[i]])
accuracy = net.accuracy(examples)
print(f"Accuracy: {accuracy}")

#probando con un dato
prediction = net.predict(X_test[2])
print(f"Desired output: {y_test[2]}")
print(f"Index of output: {prediction}")

# Modificación de la implementación de la red neuronal

# agrega el método weigths() a la clase Network, de tal forma que permita obtener los pesos de las neuronas
# agrega el método set_weights() a la clase Network, de tal forma que permite definir los pesos de las neuronas

# agrega los métodos sigmoide() y sigmoide_prime() a la clase Network
# modifica la clase Network, para que se pueda decidir qué función de activación utilizar: relu() o sigmoide()

# los métodos predict() y accuracy() de la clase Network están implementados para resolver problemas de clasificación
# modifícalos de tal manera que también se puedan utilizar con problemas de regresión

# modifica el método backpropagation() de tal manera que devuelva como resultado el array de valores de los nodos durante las épocas de entrenamiento

# una vez implementados los cambios, entrena la red neuronal del ejemplo de los apuntes
examples = []
examples.append([[0.5, 0.67, 0.5], [0.25, 0.6]])

# ejecuta la red neuronal para los datos de ejemplo de los apuntes
# comprueba los valores de los nodos y de los pesos
# los valores de los nodos tienen que ser los mismos que los de los apuntes
# los valores de los pesos son ligeramente diferentes, ¿por qué?

net = Network([3, 4, 2])


# net.set_weights([[0.1, 0.1, 0.1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3], [0.4, 0.4, 0.4], [0.5, 0.5, 0.5, 0.5], [0.6, 0.6, 0.6, 0.6], [0.6, 0.6, 0.6, 0.6]])
# net.set_weights([[0.1, 0.1, 0.1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3], [0.4, 0.4, 0.4], [0.5, 0.5, 0.5, 0.5], [0.6, 0.6, 0.6, 0.6]])
net.set_weights([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
print(net.weights())


valores_nodos = net.backpropagation(0.9, examples, 1)

print(valores_nodos)
print(net.weights())

import graphviz

def draw_graph(net):
    dot = graphviz.Digraph()
    dot.attr('node', shape='circle')
    for i, layer in enumerate(net.net):
        with dot.subgraph(name=f'cluster_{i}') as c:
            c.attr(style='invis')
            for j, node in enumerate(layer):
                c.node(f'{i},{j}')
    for i, layer in enumerate(net.net):
        for j, node in enumerate(layer):
            for input_node, weight in zip(node.inputs, node.weights):
                dot.edge(f'{i-1},{net.net[i-1].index(input_node)}', f'{i},{j}', label=f'{weight:.2f}')
    return dot

net2 = Network([3, 4, 2], 'sigmoide', True)

net2.set_weights([[0.1, 0.1, 0.1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3], [0.4, 0.4, 0.4], [0.5, 0.5, 0.5, 0.5], [0.6, 0.6, 0.6, 0.6]])

net2.weights()

draw_graph(net2)

examples = []
examples.append([[0.5, 0.67, 0.5], [0.25, 0.6]])

valores_nodos = net2.backpropagation(0.9, examples, 6000)

valores_nodos

net2.weights()

net2.predict(examples[0][0])

examples[0][1]